In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import mne
import numpy as np
from copy import deepcopy
import matplotlib.pyplot as plt
import sys; sys.path.insert(0, '../')
from esinet import util
from esinet import Simulation
from esinet import Net
from esinet.forward import create_forward_model, get_info
plot_params = dict(surface='white', hemi='both', verbose=0)

# Forward Model

In [ ]:
info = get_info()
info['sfreq'] = 100
fwd = create_forward_model(info=info)

# Simulate

In [ ]:
n_samples = 10000
settings = dict(duration_of_trial=0.2, target_snr=(0.5, 10))

sim_lstm = Simulation(fwd, info, verbose=True, settings=settings).simulate(n_samples=n_samples)
sim_dense = util.convert_simulation_temporal_to_single(sim_lstm)

sim_lstm_test = Simulation(fwd, info, verbose=True, settings=settings).simulate(n_samples=1000)
sim_dense_test = util.convert_simulation_temporal_to_single(sim_lstm_test)

# Keras-Tuner

In [16]:
import keras_tuner as kt
import tensorflow as tf
from esinet.net import build_nas_lstm

# tuner = kt.BayesianOptimization(
#     build_nas_lstm,
#     objective='val_loss',
#     max_trials=450, 
#     directory='keras-tuner',
#     project_name=f'holiday_run',
#     # max_model_size=int(2e6),  # Maximum 700k parameters per model
#     beta=2.6*2,  # exploration parameter
#     )    
tuner = kt.RandomSearch(
    build_nas_lstm,
    objective='val_loss',
    max_trials=450, 
    directory='keras-tuner',
    project_name=f'holiday_run',
    )
callbacks=[tf.keras.callbacks.EarlyStopping('val_loss', patience=15, min_delta=0.0001)]
net = Net(fwd)
x_train, y_train = net.prep_data(sim_lstm)
x_val, y_val = net.prep_data(sim_lstm_test)
tuner.search(x_train, y_train, epochs=150, validation_data=(x_val, y_val), 
    batch_size=16, callbacks=callbacks, shuffle=True)

best_model = tuner.get_best_models()[0]

Trial 7 Complete [00h 31m 16s]
val_loss: 0.17756995558738708

Best val_loss So Far: 0.17470848560333252
Total elapsed time: 02h 33m 00s

Search: Running Trial #8

Hyperparameter    |Value             |Best Value So Far 
lstm_layers       |3                 |1                 
dense_layers      |3                 |3                 
activation_out    |sigmoid           |tanh              
actvation_all     |sigmoid           |elu               
learning_rate     |0.017656          |0.015366          
Clip Value        |0.39941           |0.85608           
lstm_units_l-0    |167               |101               
dropout_lstm_l-0  |0.31976           |0.62957           
dense_units_l-0   |129               |143               
dropout_dense_l-0 |0.83085           |0.089334          
dense_units_l-1   |239               |94                
dropout_dense_l-1 |0.40118           |0.015374          
lstm_units_l-1    |74                |168               
dropout_lstm_l-1  |0.04274           |0

Epoch 1/150
625/625 [==============================] - 118s 185ms/step - loss: 0.1790 - mse: 1.0188 - mae: 0.2324 - val_loss: 0.1776 - val_mse: 1.0185 - val_mae: 0.2296
Epoch 2/150
625/625 [==============================] - 115s 184ms/step - loss: 0.1787 - mse: 1.0184 - mae: 0.2315 - val_loss: 0.1776 - val_mse: 1.0185 - val_mae: 0.2296
Epoch 3/150
625/625 [==============================] - 116s 185ms/step - loss: 0.1787 - mse: 1.0184 - mae: 0.2315 - val_loss: 0.1776 - val_mse: 1.0185 - val_mae: 0.2296
Epoch 4/150
625/625 [==============================] - 116s 186ms/step - loss: 0.1787 - mse: 1.0184 - mae: 0.2315 - val_loss: 0.1776 - val_mse: 1.0185 - val_mae: 0.2296
Epoch 5/150
625/625 [==============================] - 116s 185ms/step - loss: 0.1787 - mse: 1.0184 - mae: 0.2315 - val_loss: 0.1776 - val_mse: 1.0185 - val_mae: 0.2296
Epoch 6/150
625/625 [==============================] - 116s 185ms/step - loss: 0.1787 - mse: 1.0184 - mae: 0.2315 - val_loss: 0.1776 - val_mse: 1.0185 - va

# Visualize Keras Tuner results

In [1]:
import os
import json
import pandas as pd

base_path = 'keras-tuner/holiday_run'
folders = os.listdir(base_path)
folders = [folder for folder in folders if folder.startswith('trial_')]
hyperparams = []
scores = []
for folder in folders:
    # Load trial data
    with open(base_path + '/' + folder + '/trial.json') as json_file:
        data = json.load(json_file)
    try:
        # extract score
        scores.append( data['metrics']['metrics']['val_loss']['observations'][0]['value'][0] )
        # extract hyperparameters
        hyperparams.append( data['hyperparameters']['values'] )
    except:
        pass

dfs = []
for i, (score, hyper) in enumerate(zip(scores, hyperparams)):
    df = pd.DataFrame.from_dict(hyper, orient='index').T
    df['score'] = score
    dfs.append(df)

df = pd.concat(dfs).sort_values('score')
df


,lstm_layers,dense_layers,activation_out,actvation_all,learning_rate,Clip Value,lstm_units_l-0,dropout_lstm_l-0,dense_units_l-0,dropout_dense_l-0,dense_units_l-1,dropout_dense_l-1,lstm_units_l-1,dropout_lstm_l-1,lstm_units_l-2,dropout_lstm_l-2,dense_units_l-2,dropout_dense_l-2,score
0,1,3,tanh,elu,0.015366,0.85608,101,0.629574,143,0.089334,94,0.015374,168,0.474183,99,0.809028,25,0.0,0.174708
0,3,1,linear,linear,0.002417,0.924784,159,0.485672,27,0.797441,280,0.613588,2,0.0,2,0.0,NaN,NaN,0.175540
0,1,2,linear,linear,0.023143,0.616201,2,0.0,25,0.0,25,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.175632
0,3,1,sigmoid,sigmoid,0.010709,0.506015,77,0.553023,248,0.592647,153,0.632039,106,0.316873,72,0.599389,107,0.360547,0.177570
0,3,1,sigmoid,sigmoid,0.035422,0.930294,85,0.150921,269,0.603896,248,0.099953,91,0.357913,153,0.04135,167,0.419179,0.177570
0,2,2,tanh,sigmoid,0.009179,0.307246,172,0.592044,53,0.137369,210,0.089077,182,0.529741,170,0.621117,233,0.520172,0.177599
0,1,2,tanh,relu,0.013841,0.375405,56,0.597157,257,0.481553,41,0.076912,35,0.058192,107,0.869561,293,0.673312,0.177622
0,2,2,sigmoid,linear,0.025983,0.721979,171,0.193108,50,0.805949,225,0.363898,197,0.081307,10,0.157251,NaN,NaN,NaN


In [ ]:
df.to_excel(r'C:\Users\lukas\Dokumente\projects\esinet\keras-tuner\holidayrun.xlsx')

# Build & train LSTM network

In [ ]:
epochs = 200
patience = 20
activation_funcion = 'relu'
loss = 'huber'
dropout = 0.2
# Train
# model_params = dict(activation_function=activation_funcion, n_dense_layers=2, 
#     n_dense_units=200)
# train_params = dict(epochs=epochs, patience=patience, tensorboard=True, 
#     dropout=dropout, loss=loss, optimizer='adam', return_history=True)
# net_dense = Net(fwd, **model_params)
# _, history_dense = net_dense.fit(sim_dense, **train_params)

# LSTM v2
model_params = dict(activation_function=activation_funcion, n_lstm_layers=2, 
    n_lstm_units=75, model_type='v2')
train_params = dict(epochs=epochs, patience=patience, tensorboard=True, 
    dropout=dropout, loss=loss, optimizer=None, return_history=True, 
    batch_size=8)

net_lstm_v2 = Net(fwd, **model_params)
_, history_lstm = net_lstm_v2.fit(sim_lstm, **train_params)

# LSTM v3
# model_params = dict(activation_function=activation_funcion, n_lstm_layers=2, 
#     n_lstm_units=75, model_type='v3')
# train_params = dict(epochs=epochs, patience=patience, tensorboard=True, 
#     dropout=dropout, loss=loss, optimizer=None, return_history=True, 
#     batch_size=8)

# net_lstm_v3 = Net(fwd, **model_params)
# _, history_lstm = net_lstm_v3.fit(sim_lstm, **train_params)




In [13]:
%matplotlib qt
settings_eval = dict(duration_of_trial=0.2, target_snr=(0.5, 10))

# Simulate new data
sim_test = Simulation(fwd, info, settings=settings_eval).simulate(1)
idx = 0
# Predict sources
# prediction_dense = net_dense.predict(sim_test)
prediction_lstm = net_lstm_v2.predict(sim_test)


# Plot True Source
brain = sim_test.source_data[idx].plot(**plot_params)
brain.add_text(0.1, 0.9, 'Ground Truth', 'title')

# Plot True EEG
evoked = sim_test.eeg_data[idx].average()
evoked.plot()
evoked.plot_topomap(title='Ground Truth')
evoked = util.get_eeg_from_source(sim_test.source_data[idx], fwd, info, tmin=0.)
evoked.plot_topomap(title='Ground Truth Noiseless')


# Plot predicted source Dense
# brain = prediction_dense.plot(**plot_params)
# brain.add_text(0.1, 0.9, 'Dense', 'title')
# # Plot predicted EEG
# evoked_esi = util.get_eeg_from_source(prediction_dense, fwd, info, tmin=0.)
# evoked_esi.plot()
# evoked_esi.plot_topomap(title='Dense')

# Plot predicted source LSTM
brain = prediction_lstm.plot(**plot_params)
brain.add_text(0.1, 0.9, 'LSTM', 'title')
# Plot predicted EEG
evoked_esi = util.get_eeg_from_source(prediction_lstm, fwd, info, tmin=0.)
evoked_esi.plot()
evoked_esi.plot_topomap(title='LSTM')

error_dense = ((prediction_dense.data - sim_test.source_data[idx].data)**2).flatten()
error_lstm = ((prediction_lstm.data - sim_test.source_data[idx].data)**2).flatten()

diff = error_dense - error_lstm
relative_better_predictions = np.sum(diff>0)/ len(diff)
title = f'{relative_better_predictions*100:.1f} % of samples were better with lstm'
print(title)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

NameError: name 'prediction_dense' is not defined

  File "C:\Users\Lukas\Envs\esienv\lib\site-packages\mne\viz\_brain\_brain.py", line 1348, in _on_button_release
    self.picked_renderer = self.plotter.iren.FindPokedRenderer(x, y)
AttributeError: 'RenderWindowInteractor' object has no attribute 'FindPokedRenderer'


Using control points [3.86058518e-09 3.86058518e-09 5.35169863e-09]


  File "C:\Users\Lukas\Envs\esienv\lib\site-packages\mne\viz\_brain\_brain.py", line 1348, in _on_button_release
    self.picked_renderer = self.plotter.iren.FindPokedRenderer(x, y)
AttributeError: 'RenderWindowInteractor' object has no attribute 'FindPokedRenderer'


Using control points [1.14141424e-09 2.45816320e-09 6.30531853e-09]


# Perform predictions on test set

In [ ]:
settings = dict(duration_of_trial=0.2, target_snr=(0.5, 10))
sim_lstm_test = Simulation(fwd, info, verbose=True, settings=settings).simulate(n_samples=20)
sim_dense_test = util.convert_simulation_temporal_to_single(sim_lstm_test)

In [ ]:
# predict
models = [net_dense, net_lstm_v2, net_lstm_v3]
model_names = [model.model.name for model in models]
predictions = [model.predict(sim_lstm_test) for model in models]

## Calc Mean Localization Error

In [12]:
from esinet.evaluate import eval_mean_localization_error, eval_nmse, eval_auc, eval_mse
from scipy.spatial.distance import cdist
size = 30
pos = util.unpack_fwd(fwd)[2]
distance_matrix = cdist(pos, pos)

mean_localization_errors = []
aucs = []
nmses = []
mses = []
true_sources = np.stack([src.data for src in sim_lstm_test.source_data], axis=0)
true_sources = util.collapse(true_sources)
choice = np.random.choice(np.arange(true_sources.shape[0]), size=size, replace=False)
true_sources = true_sources[choice]
for prediction in predictions:
    predicted_sources = util.collapse(np.stack([src.data for src in prediction], axis=0))
    
    predicted_sources = predicted_sources[choice]
    
    print('mle calculation....')
    mean_localization_error = [eval_mean_localization_error(true_source, predicted_source, pos, distance_matrix=distance_matrix) for true_source, predicted_source in zip(true_sources, predicted_sources)]
    auc = [eval_auc(true_source, predicted_source, pos) for true_source, predicted_source in zip(true_sources, predicted_sources)]
    nmse = [eval_nmse(true_source, predicted_source) for true_source, predicted_source in zip(true_sources, predicted_sources)]
    mse = [eval_mse(true_source, predicted_source) for true_source, predicted_source in zip(true_sources, predicted_sources)]
    
    mean_localization_errors.append(mean_localization_error)
    aucs.append(auc)
    nmses.append(nmse)
    mses.append(mse)

aucs_far = [auc[1] for auc in aucs]
aucs_close = [auc[0] for auc in aucs]

NameError: name 'predictions' is not defined

In [ ]:
import seaborn as sns
%matplotlib qt

plt.figure()
sns.boxplot(data=nmses)
plt.title('Normalized Mean Squared Errors')

plt.figure()
sns.boxplot(data=mses)
plt.title('Mean Squared Errors')

plt.figure()
sns.boxplot(data=aucs_far)
plt.title('Far area under the curve')


plt.figure()
sns.boxplot(data=aucs_close)
plt.title('Close area under the curve')


plt.figure()
sns.boxplot(data=mean_localization_errors)
plt.title('Mean Localization Errors')
plt.tight_layout()

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(10,10))
ax.scatter(nmses[0], nmses[2], s=0.5)
ax.plot([0, 1], [0, 1], linewidth=2, color='black')
ax.set_xlim([-np.percentile(nmses[0], 5), np.percentile(nmses[0], 99)])
ax.set_ylim([-np.percentile(nmses[0], 5), np.percentile(nmses[0], 99)])
ax.set_xlabel('Dense')
ax.set_ylabel('LSTM v3')

In [ ]:
from scipy.stats import ttest_rel, wilcoxon
print('AUC_far:', ttest_rel(aucs_far[0], aucs_far[1]))
print('AUC_close:', ttest_rel(aucs_close[0], aucs_close[1]))
print('MLE:', wilcoxon(mean_localization_errors[0], mean_localization_errors[1]))
print('nMSE:', ttest_rel(nmses[0], nmses[1]))
print('MSE:', ttest_rel(mses[0], mses[1]))